In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("spam").getOrCreate()
sc=spark.sparkContext
import numpy as np
spark.conf.set('spark.executor.memory', '45G')
spark.conf.set('spark.driver.memory', '80G')
spark.conf.set('spark.driver.maxResultSize', '10G')
from pyspark.ml.feature import CountVectorizer,NGram,IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Extract words from the SMS message

In [2]:
data1=sc.textFile("/user/edureka_1152044/spam_collection.txt").map(lambda x:x.split("\n")).map(lambda x:x[0])
data1.collect()

[u'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 u'ham\tOk lar... Joking wif u oni...',
 u"spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 u'ham\tU dun say so early hor... U c already then say...',
 u"ham\tNah I don't think he goes to usf, he lives around here though",
 u"spam\tFreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, \xa31.50 to rcv",
 u'ham\tEven my brother is not like to speak with me. They treat me like aids patent.',
 u"ham\tAs per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 u'spam\tWINNER!! As a valued network customer you have been selected to receivea \xa3900 prize reward! To claim call 

Removed stop words.

In [3]:
stopwords=['ourselves','hers','between','yourself','but','again','there','about','once','during','out','very','having','with','they',
           'own','an','be','some','for','do','its','yours','such','into','of','most','itself','other','off','is','s','am','or','who',
           'as','from','him','each','the','themselves','until','below','are','we','these','your','his','through','don','nor','me','were',
           'her','more','himself','this','down','should','our','their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all',
           'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then',
           'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 
           'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against',
           'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than','u','n','y','s']

In [4]:
limiter=['.',',',':',';','?',"-","\"","\'","!",")","(","[","]","-","+","*","/","\\"]

Modify the stop words to include your custom words such as ‘-‘

In [5]:
def stop(rdd):
    line=None
    if rdd not in stopwords:
        line=rdd
    return line
def delimiter(rdd):
    for i in limiter:
        rdd=rdd.replace(i,"")
    return rdd
data2=data1.map(lambda x:x.split("\t")).map(lambda x:x[1]).map(lambda x: x.encode('utf-8')).map(delimiter)\
        .map(stop).map(lambda x:x.lower())
data2.collect()

['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat',
 'ok lar joking wif u oni',
 'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratet&cs apply 08452810075over18s',
 'u dun say so early hor u c already then say',
 'nah i dont think he goes to usf he lives around here though',
 'freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send \xc2\xa3150 to rcv',
 'even my brother is not like to speak with me they treat me like aids patent',
 'as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune',
 'winner as a valued network customer you have been selected to receivea \xc2\xa3900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only',
 'had your mobile 11 months or more u r entit

removing STOPWORDS

In [6]:
label=data1.map(lambda x:x.split("\t")[0]).collect()

In [7]:
from pyspark.sql.types import *
import pandas as pd
dfpd=pd.DataFrame(data2.collect(),columns=['value'])
dfpd['label']=label
field=[StructField("value",StringType(),True),StructField("label",StringType(),True)]
schema=StructType(field)
df=spark.createDataFrame(dfpd,schema=schema)
df.show(5)

+--------------------+-----+
|               value|label|
+--------------------+-----+
|go until jurong p...|  ham|
|ok lar joking wif...|  ham|
|free entry in 2 a...| spam|
|u dun say so earl...|  ham|
|nah i dont think ...|  ham|
+--------------------+-----+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import col,split,ltrim,regexp_replace
df1=df.withColumn("value",ltrim(col("value"))).withColumn("value",split(col("value")," ")).\
    withColumn("label",regexp_replace(col("label"),"ham","1")).\
    withColumn("label",regexp_replace(col("label"),"spam","0")).\
    withColumn("label",col("label").cast(IntegerType()))
df1.show(5)

+--------------------+-----+
|               value|label|
+--------------------+-----+
|[go, until, juron...|    1|
|[ok, lar, joking,...|    1|
|[free, entry, in,...|    0|
|[u, dun, say, so,...|    1|
|[nah, i, dont, th...|    1|
+--------------------+-----+
only showing top 5 rows



In [9]:
number_of_spam=df1.where(col("label")==0).count()
number_of_ham=df1.where(col("label")==1).count()
number_of_spam,number_of_ham

(747, 4827)

Create the features from SMS message using CountVectorizer

In [10]:
from pyspark.ml.feature import CountVectorizer,IDF
vector=CountVectorizer(inputCol='value',outputCol='feature')
idf=IDF(inputCol='feature',outputCol='features')
vector = vector.fit(df1).transform(df1)
val=idf.fit(vector).transform(vector)
val.show()

+--------------------+-----+--------------------+--------------------+
|               value|label|             feature|            features|
+--------------------+-----+--------------------+--------------------+
|[go, until, juron...|    1|(9726,[9,46,58,64...|(9726,[9,46,58,64...|
|[ok, lar, joking,...|    1|(9726,[6,47,324,4...|(9726,[6,47,324,4...|
|[free, entry, in,...|    0|(9726,[0,4,9,21,5...|(9726,[0,4,9,21,5...|
|[u, dun, say, so,...|    1|(9726,[6,24,60,14...|(9726,[6,24,60,14...|
|[nah, i, dont, th...|    1|(9726,[0,1,44,72,...|(9726,[0,1,44,72,...|
|[freemsg, hey, th...|    0|(9726,[0,2,7,12,1...|(9726,[0,2,7,12,1...|
|[even, my, brothe...|    1|(9726,[0,8,10,11,...|(9726,[0,8,10,11,...|
|[as, per, your, r...|    1|(9726,[0,12,13,51...|(9726,[0,12,13,51...|
|[winner, as, a, v...|    0|(9726,[0,2,4,16,1...|(9726,[0,2,4,16,1...|
|[had, your, mobil...|    0|(9726,[0,5,6,12,1...|(9726,[0,5,6,12,1...|
|[im, gonna, be, h...|    1|(9726,[0,1,7,23,3...|(9726,[0,1,7,23,3...|
|[six,

Split the data into train and test -decide on a strategy

In [11]:
train,test=val.randomSplit([0.7,0.3],seed=42)
train.count(),test.count()

(3848, 1726)

In [12]:
train=train.withColumn("label",col("label").cast(IntegerType())).select("features","label")
test=test.withColumn("label",col("label").cast(IntegerType())).select("features","label")
test.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(9726,[0,2,3,4,12...|    0|
|(9726,[95,182,312...|    1|
|(9726,[16,219,679...|    0|
|(9726,[2,8,12,16,...|    0|
|(9726,[4,9,15,32,...|    1|
+--------------------+-----+
only showing top 5 rows



Use logistic regression and check the accuracy

In [13]:
denominator=test.count()
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    for j in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        print(i,"\t",j)
        lr=LogisticRegression(maxIter=20,regParam=i, elasticNetParam=j)
        result=lr.fit(train).transform(test).select("label","prediction")
        numerator=result.where(col("label")==col("prediction")).count()
        accuracy=float(numerator)/float(denominator)
        print(accuracy)

(0.1, '\t', 0.1)
0.957705677868
(0.1, '\t', 0.2)
0.943800695249
(0.1, '\t', 0.3)
0.924681344148
(0.1, '\t', 0.4)
0.906141367323
(0.1, '\t', 0.5)
0.891657010429
(0.1, '\t', 0.6)
0.886442641947
(0.1, '\t', 0.7)
0.877172653534
(0.1, '\t', 0.8)
0.869640787949
(0.1, '\t', 0.9)
0.86674391657
(0.2, '\t', 0.1)
0.933951332561
(0.2, '\t', 0.2)
0.897450753187
(0.2, '\t', 0.3)
0.877172653534
(0.2, '\t', 0.4)
0.865585168019
(0.2, '\t', 0.5)
0.863267670915
(0.2, '\t', 0.6)
0.863267670915
(0.2, '\t', 0.7)
0.863267670915
(0.2, '\t', 0.8)
0.863267670915
(0.2, '\t', 0.9)
0.863267670915
(0.3, '\t', 0.1)
0.905561993048
(0.3, '\t', 0.2)
0.873696407879
(0.3, '\t', 0.3)
0.863267670915
(0.3, '\t', 0.4)
0.863267670915
(0.3, '\t', 0.5)
0.863267670915
(0.3, '\t', 0.6)
0.863267670915
(0.3, '\t', 0.7)
0.863267670915
(0.3, '\t', 0.8)
0.863267670915
(0.3, '\t', 0.9)
0.863267670915
(0.4, '\t', 0.1)
0.88180764774
(0.4, '\t', 0.2)
0.863267670915
(0.4, '\t', 0.3)
0.863267670915
(0.4, '\t', 0.4)
0.863267670915
(0.4, '\t'

Try to use a Random Forestclassifier and see if it increases the accuracy

In [14]:
denominator=test.count()
for i in range(1,10):
    rfc=RandomForestClassifier(numTrees=i)
    result=rfc.fit(train).transform(test).select("label","prediction")
    numerator=result.where(col("label")==col("prediction")).count()
    accuracy=float(numerator)/float(denominator)
    print("{0} : {1}".format(i,accuracy))

1 : 0.946697566628
2 : 0.884704519119
3 : 0.870799536501
4 : 0.898030127462
5 : 0.868482039397
6 : 0.874275782155
7 : 0.867323290846
8 : 0.866164542294
9 : 0.864426419467


Introduce bi-gram and tri-gram and note the change in accuracy

In [15]:
def logistics(i,train,test,log,ngram,vector,idf,df1):
    denominator=test.count()
    train,test=df1.randomSplit([0.7,0.3],2)
    pipeline=Pipeline(stages=[ngram,vector,idf,log])
    model=pipeline.fit(train)
    result=model.transform(test).select("label","prediction")
    numerator=result.where(col("label")==col("prediction")).count()
    acc=float(numerator)/denominator
    print(i,acc)

In [16]:
import numpy
train,test=df1.randomSplit([0.7,0.3],3)
num=[]
for i in range(0,10):
    num.append(i*0.1)
ngram=NGram(n=2,inputCol='value',outputCol='grams')
vector=CountVectorizer(inputCol='grams',outputCol='vector')
idf=IDF(inputCol='vector',outputCol='features')
for i in num:
    log=LogisticRegression(maxIter=20,regParam=i)
    logistics(i,train,test,log,ngram,vector,idf,df1)

(0.0, 0.9449152542372882)
(0.1, 0.9394673123486683)
(0.2, 0.9388619854721549)
(0.30000000000000004, 0.9358353510895884)
(0.4, 0.9352300242130751)
(0.5, 0.9340193704600485)
(0.6000000000000001, 0.9340193704600485)
(0.7000000000000001, 0.9328087167070218)
(0.8, 0.9303874092009685)
(0.9, 0.9303874092009685)


In [17]:
train,test=df1.randomSplit([0.7,0.3],3)
num=[]
for i in range(0,10):
    num.append(i*0.1)
ngram=NGram(n=3,inputCol='value',outputCol='grams')
vector=CountVectorizer(inputCol='grams',outputCol='vector')
idf=IDF(inputCol='vector',outputCol='features')
for i in num:
    log=LogisticRegression(maxIter=20,regParam=i)
    logistics(i,train,test,log,ngram,vector,idf,df1)

(0.0, 0.9497578692493946)
(0.1, 0.9303874092009685)
(0.2, 0.927360774818402)
(0.30000000000000004, 0.927360774818402)
(0.4, 0.9261501210653753)
(0.5, 0.9255447941888619)
(0.6000000000000001, 0.9255447941888619)
(0.7000000000000001, 0.9255447941888619)
(0.8, 0.9243341404358354)
(0.9, 0.923728813559322)


In [18]:
def random(i,train,test,log,ngram,vector,df1):
    denominator=test.count()
    train,test=df1.randomSplit([0.7,0.3],2)
    pipeline=Pipeline(stages=[ngram,vector,log])
    model=pipeline.fit(train)
    result=model.transform(test).select("label","prediction")
    numerator=result.where(col("label")==col("prediction")).count()
    acc=float(numerator)/denominator
    print(i,acc)